In [1]:
PYTHONIOENCODING='utf-8'
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib, sys, os
import re
import time
import codecs
import json
import traceback
import datetime
import pickle as pk
# chromedriver = "./chromedriver"
# os.environ["webdriver.chrome.driver"] = chromedriver
# driver = webdriver.Chrome(chromedriver)

In [2]:
hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',}

In [3]:
opener = urllib.request.build_opener()

In [8]:
def get_year_info():
    req = urllib.request.Request('https://www.aclweb.org/anthology/venues/acl/', headers=hdr)
    response = opener.open(req)
    bsObj = BeautifulSoup(response, "html")
    rows = bsObj.find_all('div', { "class" : "row" })
    all_volumes = []
    for row in rows:
        year = int(row.h4.text)
        if year > 2009:
            continue
        ilis = row.find_all('li')
        for li in ilis:
            procedding = li.a.text
            if 'Tutorials' in procedding:
                continue
            url = 'https://www.aclweb.org' + li.a['href']
            all_volumes.append([int(year), procedding, url])
    return all_volumes

In [9]:
all_volumes = get_year_info()

In [10]:
all_volumes

[[2009,
  'Proceedings of the Joint Conference of the 47th Annual Meeting of the ACL and the 4th International Joint Conference on Natural Language Processing of the AFNLP',
  'https://www.aclweb.org/anthology/volumes/proceedings-of-the-joint-conference-of-the-47th-annual-meeting-of-the-acl-and-the-4th-international-joint-conference-on-natural-language-processing-of-the-afnlp/'],
 [2009,
  'Proceedings of the ACL-IJCNLP 2009 Conference Short Papers',
  'https://www.aclweb.org/anthology/volumes/proceedings-of-the-acl-ijcnlp-2009-conference-short-papers/'],
 [2009,
  'Proceedings of the ACL-IJCNLP 2009 Student Research Workshop',
  'https://www.aclweb.org/anthology/volumes/proceedings-of-the-acl-ijcnlp-2009-student-research-workshop/'],
 [2009,
  'Proceedings of the ACL-IJCNLP 2009 Software Demonstrations',
  'https://www.aclweb.org/anthology/volumes/proceedings-of-the-acl-ijcnlp-2009-software-demonstrations/'],
 [2009,
  'Tutorial Abstracts of ACL-IJCNLP 2009',
  'https://www.aclweb.org

In [48]:
def get_all_cite(article_url):
    driver.get(article_url)
    atl_page = driver.page_source.encode('utf-8')
    atl_obj = BeautifulSoup(atl_page)
    try:
        cited_section = atl_obj.find('div', {'class':'gs_ri'}).find('div', {'class':'gs_fl'}).find_all('a')[2]
        total_citation = int(cited_section.text.split()[-1])
        cited_info_url = 'https://scholar.google.com.sg'+cited_section['href']+'&scipsc=&as_ylo=2014&as_yhi=2019'
#         total_citation = int(atl_obj.find('div', {'class':'gs_ri'}).find('div', {'class':'gs_fl'}).find_all('a')[2].text.split()[-1])
#         cited_info_url = 'https://scholar.google.com.sg'+atl_obj.find('div', {'class':'gs_ri'}).find('div', {'class':'gs_fl'}).find_all('a')[2]['href']+'&scipsc=&as_ylo=2014&as_yhi=2019'
    except IndexError:
        total_citation = 0
        cited_info_url = 0
    except ValueError:
        total_citation = 0
        cited_info_url = 0
    except AttributeError:
        total_citation = 0
        cited_info_url = 0
#     cited_section = atl_obj.find('div', {'class':'gs_ri'}).find('div', {'class':'gs_fl'}).find_all('a')[2]
#     total_citation = int(cited_section.text.split()[-1])
#     cited_info_url = 'https://scholar.google.com.sg'+cited_section['href']
    return total_citation, cited_info_url

In [15]:
def get_volume_info(year_url):
    vlm_req = urllib.request.Request(year_url, headers=hdr)
    vlm_page = opener.open(vlm_req)
    vlm_obj = BeautifulSoup(vlm_page)
    articles = vlm_obj.find('div', {'id': 'main-container'}).section.find_all('p')[1:]
    articles_info = []
    for article in articles:
        atl_url = article.span.a['href']
        title = article.strong.text
        googlescl = 'https://scholar.google.com.sg/scholar?hl=en&as_sdt=0%2C5&q='+title.replace(' ', '+')+'&btnG='
        atl_ctn, cited_info = get_all_cite(googlescl)
        atl_authors = ', '.join([itm.text for itm in article.find_all('a') if 'anthology/people/' in itm['href']])
        articles_info.append([title, atl_url, atl_authors, atl_ctn, cited_info])
        print([title, atl_url, atl_authors, atl_ctn, cited_info])
    return articles_info

In [49]:
chromedriver = "./chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [50]:
all_info84 = []
for idx, itm in enumerate(all_volumes):
    year = itm[0]
    if year >= 1985:
        continue
    procedding = itm[1]
    info_vo = [year, procedding, get_volume_info(itm[-1])]
    with open('./acl/%d_%d'%(year, idx), 'wb') as filout:
        pk.dump(info_vo, filout)
    all_info84.append(info_vo)

['Multilingual Text Processing in a Two-Byte Code', 'https://www.aclweb.org/anthology/P84-1001', 'Lloyd B. Anderson', 0, 0]
['CONVEYING IMPLICIT CONTENT IN NARRATIVE SUMMARIES', 'https://www.aclweb.org/anthology/P84-1002', 'Malcolm E. Cook, Wendy G. Lehnert, David D. McDonald', 0, 0]
['TRANSFORMING ENGLISH INTERFACES TO OTHER NATURAL LANGUAGES: AN EXPERIMENT WITH PORTUGUESE', 'https://www.aclweb.org/anthology/P84-1003', 'GABRIEL PEREIRA LOPES', 0, 0]
['UN OUTIL MULTIDIMENSIONNEL DE L’ANALYSE DU DISCOURS', 'https://www.aclweb.org/anthology/P84-1004', 'J. CHAUCHÉ', 0, 0]
['A STOCHASTIC APPROACH TO SENTENCE PARSING', 'https://www.aclweb.org/anthology/P84-1005', 'Tetsunosuke Fujisaki', 0, 0]
['BOUNDED CONTEXT PARSING AND EASY LEARNABIITY', 'https://www.aclweb.org/anthology/P84-1006', 'Robert C. Berwick', 0, 0]
['THE REPRESENTATION OF CONSTITUENT STRUCTURES FOR FINITE-STATE PARSING', 'https://www.aclweb.org/anthology/P84-1007', 'D. Terence Langendoen, Yedidyah Langsam', 0, 0]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=75.0.3770.80)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Linux 4.15.0-51-generic x86_64)


In [ ]:
def ranking(voinfo):
    _25vo = [itm for itm in voinfo if itm[0] <= 1994]
    _25vo = [articl for itm in _25vo for articl in itm[-1]]
    _10vo = [articl for itm in voinfo for articl in itm[-1]]
    _25vo = sorted(_25vo, key=lamada k:k[-2], reverse=True)
    _10vo = sorted(_10vo, key=lamada k:k[-2], reverse=True)
    print('Top 20 in 25')
    print()
    print(_25vo[:20])
    print('Top 10 in 10')
    

In [69]:
def noce_print(varlist):
    print('Article Title\tURL\tAuthors\tCitation Number\tRecent5')
    for itm in varlist:
        print('%s\t%s\t%s\t%d\t'%(itm[0], itm[1], itm[2], itm[3]))

In [70]:
noce_print(sorted(year10, key=lambda k:k[-1], reverse=True))

Article Title	URL	Authors	Citation Number	Recent5
Bleu: a Method for Automatic Evaluation of Machine Translation	https://www.aclweb.org/anthology/P02-1040	Kishore Papineni, Salim Roukos, Todd Ward, Wei-Jing Zhu	9077	
Learning Bigrams from Unigrams	https://www.aclweb.org/anthology/P08-1075	Xiaojin Zhu, Andrew B. Goldberg, Michael Rabbat, Robert Nowak	8170	
WHAT NOT TO SAY	https://www.aclweb.org/anthology/P84-1071	Jan Fornell	7812	
Thumbs Up or Thumbs Down? Semantic Orientation Applied to Unsupervised Classification of Reviews	https://www.aclweb.org/anthology/P02-1053	Peter Turney	5630	
ACL IN 1977	https://www.aclweb.org/anthology/P82-1024	Paul G. Chapin	5339	
Word Association Norms, Mutual Information, and Lexicography	https://www.aclweb.org/anthology/P89-1010	Kenneth Ward Church, Patrick Hanks	4846	
Moses: Open Source Toolkit for Statistical Machine Translation	https://www.aclweb.org/anthology/P07-2045	Philipp Koehn, Hieu Hoang, Alexandra Birch, Chris Callison-Burch, Marcello Federico,

In [71]:
noce_print(sorted(year25, key=lambda k:k[-1], reverse=True))

Article Title	URL	Authors	Citation Number	Recent5
WHAT NOT TO SAY	https://www.aclweb.org/anthology/P84-1071	Jan Fornell	7812	
ACL IN 1977	https://www.aclweb.org/anthology/P82-1024	Paul G. Chapin	5339	
Word Association Norms, Mutual Information, and Lexicography	https://www.aclweb.org/anthology/P89-1010	Kenneth Ward Church, Patrick Hanks	4846	
APPLICATIONS	https://www.aclweb.org/anthology/P79-1022	DAVID G. HAYS	4311	
VERB SEMANTICS AND LEXICAL SELECTION	https://www.aclweb.org/anthology/P94-1019	Zhibiao Wu, Martha Palmer	3563	
PARSING	https://www.aclweb.org/anthology/P81-1022	Ralph Grishman	3193	
Parsing	https://www.aclweb.org/anthology/P80-1024	W. A. Martin	3193	
MY TERM	https://www.aclweb.org/anthology/P82-1020	Winfred P. Lehmann	2519	
MACHINE-READABLE DICTIONARIES	https://www.aclweb.org/anthology/P84-1093	Donald E. Walker	2332	
Machine Translation will not Work	https://www.aclweb.org/anthology/P86-1040	Martin Kay	2102	
LINGUISTIC AND COMPUTATIONAL SEMANTICS	https://www.aclweb.org/anth

In [72]:
year10

[['Proceedings of the Joint Conference of the 47th Annual Meeting of the ACL and the 4th International Joint Conference on Natural Language Processing of the AFNLP',
  'https://www.aclweb.org/anthology/P09-1000',
  'Keh-Yih Su, Jian Su, Janyce Wiebe, Haizhou Li',
  5],
 ['Heterogeneous Transfer Learning for Image Clustering via the SocialWeb',
  'https://www.aclweb.org/anthology/P09-1001',
  'Qiang Yang, Yuqiang Chen, Gui-Rong Xue, Wenyuan Dai, Yong Yu',
  131],
 ['Investigations on Word Senses and Word Usages',
  'https://www.aclweb.org/anthology/P09-1002',
  'Katrin Erk, Diana McCarthy, Nicholas Gaylord',
  61],
 ['A Comparative Study on Generalization of Semantic Roles in FrameNet',
  'https://www.aclweb.org/anthology/P09-1003',
  'Yuichiroh Matsubayashi, Naoaki Okazaki, Jun’ichi Tsujii',
  21],
 ['Unsupervised Argument Identification for Semantic Role Labeling',
  'https://www.aclweb.org/anthology/P09-1004',
  'Omri Abend, Roi Reichart, Ari Rappoport',
  50],
 ['Brutus: A Semantic 

In [73]:
year25

[['Discourse Obligations in Dialogue Processing',
  'https://www.aclweb.org/anthology/P94-1001',
  'David R. Traum, James F. Allen',
  327],
 ['MULTI-PARAGRAPH SEGMENTATION EXPOSITORY TEXT',
  'https://www.aclweb.org/anthology/P94-1002',
  'Marti A. Hearst',
  836],
 ['PRIORITY UNION AND GENERALIZATION IN DISCOURSE GRAMMARS',
  'https://www.aclweb.org/anthology/P94-1003',
  'Claire Grover, Chris Brew, Suresh Manandhar, Marc Moens',
  48],
 ['HIDDEN UNDERSTANDING MODELS OF NATURAL LANGUAGE',
  'https://www.aclweb.org/anthology/P94-1004',
  'Scott Miller, Robert Bobrow, Robert Ingria, Richard Schwartz',
  133],
 ['From Strings to Trees to Strings to Trees (Abstract)',
  'https://www.aclweb.org/anthology/P94-1005',
  'Aravind K. Joshi',
  512],
 ['INTENTIONS AND INFORMATION IN DISCOURSE',
  'https://www.aclweb.org/anthology/P94-1006',
  'Nicholas Asher, Alex Lascarides',
  84],
 ['GENERATING PRECONDITION EXPRESSIONS IN INSTRUCTIONAL TEXT',
  'https://www.aclweb.org/anthology/P94-1007',
  